In [1]:
import open3d as o3d
import numpy as np
import copy
import math

In [2]:
# chair_mesh = o3d.io.read_triangle_mesh(".\\data\\chair.stl")

In [3]:
# o3d.visualization.draw_geometries([chair_mesh])

In [4]:
# chair_pcd = chair_mesh.sample_points_uniformly(number_of_points=5000)

In [5]:
# o3d.visualization.draw_geometries([chair_pcd])

## ICP

### helper visulization

In [6]:
# add aixs
aix_points = [[0, 0, 0],
              [400, 0, 0],
              [0, 600, 0],
              [0, 0, 800],
              ]
aix_lines = [[0, 1],
             [0, 2],
             [0, 3]]
colors = [[0, 0, 0], [0,0,1],[0,0,0]]
aix_line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(aix_points),
    lines=o3d.utility.Vector2iVector(aix_lines),
)
aix_line_set.colors = o3d.utility.Vector3dVector(colors)

In [7]:
def draw_original_compare(source, target, name):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([0, 1, 0])
    target_temp.paint_uniform_color([1, 0, 0])
    o3d.visualization.draw_geometries([source_temp, target_temp, aix_line_set],name)


def draw_registration_result(source, target, transformation,name):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([0, 1, 0])
    target_temp.paint_uniform_color([1, 0, 0])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp, aix_line_set], name)

### source and target

source

In [8]:
source = o3d.io.read_point_cloud("./data/chair.ply")
o3d.visualization.draw_geometries([source])

target

In [9]:
target = o3d.io.read_point_cloud("./data/chair.ply")
# trans_true = np.asarray([[1.,   0,         0,         .5],
#                         [0,   1.,         0,         .5],
#                         [0,              0,    1.0000,   0.2],
#                         [0,              0,         0,     1.0000]])
alpha = math.pi / 10
trans_true = np.asarray([[math.cos(alpha),       -math.sin(alpha),         0,        2],
                         [math.sin(alpha),        math.cos(alpha),         0,        3],
                         [0,        0,         1,        4],
                        [ 0,        0,         0,        1]])
target.transform(trans_true)
# o3d.visualization.draw_geometries([target, aix_line_set])

geometry::PointCloud with 250002 points.

draw original comparison

In [10]:
draw_original_compare(source, target, 'source: green; target : red ')

In [11]:
# draw_registration_result(source, target, trans_true, 'idelistic result')

In [12]:
threshold = 50
# trans_init = np.asarray([[1,   0,         0,         0],
#                         [0,   1,         0,         0],
#                         [0,              0,    1.0000,         0],
#                         [0,              0,         0,     1.0000]])
trans_init = np.asarray([[math.cos(alpha),       -math.sin(alpha),         0,        0],
                         [math.sin(alpha),        math.cos(alpha),         0,        3],
                         [0,        0,         1,        4],
                        [ 0,        0,         0,        1]])
draw_registration_result(source, target, trans_init, 'init matrix result')

### initial alignment

In [13]:
evaluation = o3d.registration.evaluate_registration(source, target, threshold, trans_init)
print(evaluation)

registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=1.729926e+00, and correspondence_set size of 250002
Access transformation to get result.


###  point-to-point ICP

In [14]:
print("Apply point-to-point ICP")
reg_p2p = o3d.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.registration.TransformationEstimationPointToPoint(),
        o3d.registration.ICPConvergenceCriteria(max_iteration = 300))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)

Apply point-to-point ICP
registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=5.973004e-13, and correspondence_set size of 250002
Access transformation to get result.
Transformation is:
[[ 9.51056516e-01 -3.09016994e-01 -4.88100354e-16  2.00000000e+00]
 [ 3.09016994e-01  9.51056516e-01  5.09289915e-16  3.00000000e+00]
 [-4.80120027e-16  5.35578609e-16  1.00000000e+00  4.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [15]:
draw_registration_result(source, target, reg_p2p.transformation, 'p2p')

### point-to-plane ICP

In [16]:
source.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=1, max_nn=30))
target.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=1, max_nn=30))

reg_p2l = o3d.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.registration.TransformationEstimationPointToPlane(),
        o3d.registration.ICPConvergenceCriteria(max_iteration = 100))
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)


registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=6.317049e-14, and correspondence_set size of 250002
Access transformation to get result.
Transformation is:
[[ 9.51056516e-01 -3.09016994e-01 -1.97821959e-18  2.00000000e+00]
 [ 3.09016994e-01  9.51056516e-01  7.96093640e-18  3.00000000e+00]
 [-6.21114418e-19 -8.15919278e-18  1.00000000e+00  4.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [17]:
draw_registration_result(source, target, reg_p2l.transformation,'p2l')